In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = load_dataset('poloclub/diffusiondb', '2m_first_10k')
df = dataset['train'].to_pandas()
df.drop_duplicates(subset=['prompt'], inplace=True)
df['image'] = df['image'].apply(lambda x: x['path'])

df = df.sample(n=2000, random_state=42).reset_index(drop=True)

df.reset_index(inplace=True, drop=True)

df.head(2)

  0%|          | 0/1 [00:00<?, ?it/s]

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,/root/.cache/huggingface/datasets/downloads/ex...,"classic oil painting, sci fi beautiful landsca...",3578732311,50,12.0,k_lms,1280,512,26da12cd3526e1b10319bb5d94ea516909c1cb4a5b77da...,2022-08-18 20:28:00+00:00,0.059204,0.000319
1,/root/.cache/huggingface/datasets/downloads/ex...,"3 panel scene, anime girl sleeps inside a cozy...",1787203390,50,5.0,k_lms,512,1024,56b9f823a65b1259d7a469d0e0726722e6f87297384c8f...,2022-08-20 02:43:00+00:00,0.183237,0.000596


In [ ]:
df.shape

(2000, 12)

In [ ]:
labels = pd.read_csv('labels.csv')
df = df.merge(labels, on='prompt', how='inner')
df.head(1)

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,...,phrase_1,phrase_2,phrase_3,phrase_4,phrase_5,phrase_6,phrase_7,first_seven,cs_prompt_open,cs_prompt_seven
0,/root/.cache/huggingface/datasets/downloads/ex...,"classic oil painting, sci fi beautiful landsca...",3578732311,50,12.0,k_lms,1280,512,26da12cd3526e1b10319bb5d94ea516909c1cb4a5b77da...,2022-08-18 20:28:00+00:00,...,a man standing on a rock in a mountain valley,8 k matte painting,8k matte painting,matte fantasy painting,4 k matte painting,luminescent matte painting,high fantasy matte painting,a man standing on rock in mountain valley 8 k ...,0.392391,0.330291


In [ ]:
labels.head(1)

,prompt,open_clip,clip_inter,phrase_1,phrase_2,phrase_3,phrase_4,phrase_5,phrase_6,phrase_7,first_seven,cs_prompt_open,cs_prompt_seven
0,"classic oil painting, sci fi beautiful landsca...",a painting of a mountain range with a body of ...,['a man standing on a rock in a mountain valle...,a man standing on a rock in a mountain valley,8 k matte painting,8k matte painting,matte fantasy painting,4 k matte painting,luminescent matte painting,high fantasy matte painting,a man standing on rock in mountain valley 8 k ...,0.392391,0.330291


## Column Explanations

| column | meaning |
| --- | --- |
| open_clip | open clip response |
| clip_inter | clip interrogator full response, list format |
| phrase_1 | first phrase from clip interrogator |
| phrase_2 | second phrase from clip interrogator |
| phrase_3 | third phrase from clip interrogator |
| phrase_4 | fourth phrase from clip interrogator |
| phrase_5 | fifth phrase from clip interrogator |
| phrase_6 | sixth phrase from clip interrogator |
| phrase_7 | seventh phrase from clip interrogator |
|first_seven | all 7 phrase concatenated as a string, dropped duplicate words |
| cs_prompt_open | cosine similarity of original prompt and open clip response |
| cs_prompt_seven | cosine similarity of prompt and "first_seven" of clip interrogator|



## Test Train Split

In [ ]:
train = df.iloc[0:1500]
test = df.iloc[1500:2000]

train.shape, test.shape

((1500, 24), (500, 24))

In [ ]:
train.head(1)

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,...,phrase_1,phrase_2,phrase_3,phrase_4,phrase_5,phrase_6,phrase_7,first_seven,cs_prompt_open,cs_prompt_seven
0,/root/.cache/huggingface/datasets/downloads/ex...,"classic oil painting, sci fi beautiful landsca...",3578732311,50,12.0,k_lms,1280,512,26da12cd3526e1b10319bb5d94ea516909c1cb4a5b77da...,2022-08-18 20:28:00+00:00,...,a man standing on a rock in a mountain valley,8 k matte painting,8k matte painting,matte fantasy painting,4 k matte painting,luminescent matte painting,high fantasy matte painting,a man standing on rock in mountain valley 8 k ...,0.392391,0.330291


In [ ]:
test.head(1)

,image,prompt,seed,step,cfg,sampler,width,height,user_name,timestamp,...,phrase_1,phrase_2,phrase_3,phrase_4,phrase_5,phrase_6,phrase_7,first_seven,cs_prompt_open,cs_prompt_seven
1500,/root/.cache/huggingface/datasets/downloads/ex...,"lemonbrat studios bat fursuit, fursuiter, cand...",1758474385,50,10.0,k_lms,512,640,eaa41f839ff05c773bb8a0907424edbfeedf3ae4ccce84...,2022-08-16 19:17:00+00:00,...,a person in a costume with a cat mask on,fursuit,fursuit!!!!,furry character portrait,headshot of young female furry,furry character,macro furry,a person in costume with cat mask on fursuit f...,0.228366,0.474969
